In [13]:
import pandas as pd
from cfg import DATA_PATH, ADDRESSES_CSV, ONEMAP_API_URL

In [70]:
def transform_districts(districts: pd.DataFrame):
    # Iterate through each row of districts df
    new_data = []

    for index, row in districts.iterrows():
        postal_sector = (row["Postal Sector"])
        # Remove whitespace from entire string and split
        postal_sectors = postal_sector.replace(" ", "").split(",")

        for ps in postal_sectors:
            new_data.append({"Postal District": str(row["Postal District"]), "Postal Sector": str(ps)})

    return pd.DataFrame(new_data, columns=["Postal District", "Postal Sector"])

In [71]:
transform_districts(districts)

,Postal District,Postal Sector
0,1,01
1,1,02
2,1,03
3,1,04
4,1,05
...,...,...
77,26,78
78,27,75
79,27,76
80,28,79


In [79]:
# function to get district from postal code from districts table
def get_district_from_postal(postal: str, districts: pd.DataFrame):
    postal_sector = str(postal)[:2]
    print(postal_sector, districts)
    try:
        district = districts[districts["Postal Sector"] == postal_sector]["Postal District"].values[0]
    except IndexError:
        district = None
    return district

In [81]:
get_district_from_postal("56sdf", transform_districts(districts))

56    Postal District Postal Sector
0                1            01
1                1            02
2                1            03
3                1            04
4                1            05
..             ...           ...
77              26            78
78              27            75
79              27            76
80              28            79
81              28            80

[82 rows x 2 columns]


'20'

In [62]:
districts_transformed = transform_districts(districts)

In [69]:
districts_transformed["Postal Sector"].unique()

array(['01', ' 02', ' 03', ' 04', ' 05', ' 06', '07', ' 08', '14', ' 15',
       ' 16', '09', ' 10', '11', ' 12', ' 13', '17', '', '18', ' 19',
       '20', ' 21', '22', ' 23', '24', ' 25', ' 26', ' 27', '28', ' 29',
       ' 30', '31', ' 32', ' 33', '34', ' 35', ' 36', ' 37', '38', ' 39',
       ' 40', ' 41', '42', ' 43', ' 44', ' 45', '46', ' 47', ' 48', '49',
       ' 50', ' 81', '51', ' 52', '53', ' 54', ' 55', ' 82', '56', ' 57',
       '58', ' 59', '60', ' 61', ' 62', ' 63', ' 64', '65', ' 66', ' 67',
       ' 68', '69', ' 70', ' 71', '72', ' 73', '77', ' 78', '75', ' 76',
       '79', ' 80'], dtype=object)

In [67]:
districts_transformed[districts_transformed["Postal Sector"] == " 02"]   

,Postal District,Postal Sector
1,1,02


In [17]:
pd.read_excel("C:/Users/valen/Documents/GitHub/bt4301/source/airflow/dags/data/districts.xlsx")

,Postal District,Postal Sector,General Location
0,1,"01, 02, 03, 04, 05, 06","Raffles Place, Cecil, Marina, People’s Park"
1,2,"07, 08","Anson, Tanjong Pagar"
2,3,"14, 15, 16","Queenstown, Tiong Bahru"
3,4,"09, 10","Telok Blangah, Harbourfront"
4,5,"11, 12, 13","Pasir Panjang, Hong Leong Garden, Clementi New..."
5,6,"17,","High Street, Beach Road (part)"
6,7,"18, 19","Middle Road, Golden Mile"
7,8,"20, 21",Little India
8,9,"22, 23","Orchard, Cairnhill, River Valley"
9,10,"24, 25, 26, 27","Ardmore, Bukit Timah, Holland Road, Tanglin"


In [44]:
districts = (pd.read_excel("C:/Users/valen/Documents/GitHub/bt4301/source/airflow/dags/data/districts.xlsx"))

In [45]:
districts

,Postal District,Postal Sector,General Location
0,1,"01, 02, 03, 04, 05, 06","Raffles Place, Cecil, Marina, People’s Park"
1,2,"07, 08","Anson, Tanjong Pagar"
2,3,"14, 15, 16","Queenstown, Tiong Bahru"
3,4,"09, 10","Telok Blangah, Harbourfront"
4,5,"11, 12, 13","Pasir Panjang, Hong Leong Garden, Clementi New..."
5,6,"17,","High Street, Beach Road (part)"
6,7,"18, 19","Middle Road, Golden Mile"
7,8,"20, 21",Little India
8,9,"22, 23","Orchard, Cairnhill, River Valley"
9,10,"24, 25, 26, 27","Ardmore, Bukit Timah, Holland Road, Tanglin"


In [39]:
# iterate through each row of districts df

# create new dataframe with 2 columns (postal district, postal sector)
new_dataframe = pd.DataFrame(columns=["Postal District", "Postal Sector"])

for index, row in districts.iterrows():
    postal_sector = row["Postal Sector"]
    # remove whitespace and split
    postal_sectors = postal_sector.strip().split(",")

    for ps in postal_sectors:
        new_dataframe = new_dataframe.append({"Postal District": row["Postal District"], "Postal Sector": ps}, ignore_index=True)

new_dataframe




C:\Users\valen\AppData\Local\Temp\ipykernel_39804\1622617137.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_dataframe = new_dataframe.append({"Postal District": row["Postal District"], "Postal Sector": ps}, ignore_index=True)
C:\Users\valen\AppData\Local\Temp\ipykernel_39804\1622617137.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_dataframe = new_dataframe.append({"Postal District": row["Postal District"], "Postal Sector": ps}, ignore_index=True)
C:\Users\valen\AppData\Local\Temp\ipykernel_39804\1622617137.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_dataframe = new_dataframe.append({"Postal District": row["Postal District"], "Postal Sector": ps}, ignore_index=True)
C:\Users\valen\AppData\Lo

,Postal District,Postal Sector
0,1,01
1,1,02
2,1,03
3,1,04
4,1,05
...,...,...
77,26,78
78,27,75
79,27,76
80,28,79


In [40]:
new_dataframe

,Postal District,Postal Sector
0,1,01
1,1,02
2,1,03
3,1,04
4,1,05
...,...,...
77,26,78
78,27,75
79,27,76
80,28,79


In [42]:
new_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Postal District  82 non-null     object
 1   Postal Sector    82 non-null     object
dtypes: object(2)
memory usage: 1.4+ KB
